In [ ]:
import pygsheets
import difflib
import re

In [ ]:
def client_auth(use_pyg=True):
    # authorization
    # using pygsheets
    if use_pyg:
        gc = pygsheets.authorize(service_file="client_secret.json")
        return gc
    else:
        scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
        client = gspread.authorize(creds)
        return client

In [2]:
def fix_names(name):
    return name.split(' ')

def fix_depts(dept):
    return 